In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# #from keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

2022-04-01 11:03:33.926683: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-01 11:03:33.926724: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/home/vimalkumar/Documents/Stewart/CNN/Angle_cnn/train_1/train'
#valid_path = '/home/vimalkumar/Documents/softco/CNN/CNN/test'

In [3]:
vgg = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)


2022-04-01 11:04:12.581968: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-01 11:04:12.581994: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-01 11:04:12.582008: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2022-04-01 11:04:12.633196: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [5]:
folders = glob('/home/vimalkumar/Documents/Stewart/CNN/Angle_cnn/train_1/train/*')


In [6]:
x = Flatten()(vgg.output)


In [7]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [8]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [9]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                  width_shift_range=0.2,
                                height_shift_range=0.2,
                                brightness_range=(0.2,0.8),
                                  fill_mode='nearest',
                                  validation_split = 0.2)
                           

#test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                subset = 'training')

test_set = train_datagen.flow_from_directory(train_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                           subset = 'validation')

Found 4132 images belonging to 5 classes.
Found 1031 images belonging to 5 classes.


In [11]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  #steps_per_epoch=len(training_set),
  #validation_steps=len(test_set)
)

/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-04-01 11:04:24.582935: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2022-04-01 11:04:28.100028: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 205520896 exceeds 10% of free system memory.
2022-04-01 11:04:28.675039: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 205520896 exceeds 10% of free system memory.
2022-04-01 11:04:29.076768: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 102760448 exceeds 10% of free system memory.
2022-04-01 11:04:29.225201: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 102760448 exceeds 10% of free system memory.


  1/259 [..............................] - ETA: 27:42 - loss: 1.8390 - accuracy: 0.1250

2022-04-01 11:04:32.517993: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 205520896 exceeds 10% of free system memory.


259/259 [==============================] - 1258s 5s/step - loss: 0.7488 - accuracy: 0.6975 - val_loss: 0.7223 - val_accuracy: 0.7081
Epoch 2/5
259/259 [==============================] - 1013s 4s/step - loss: 0.5757 - accuracy: 0.7834 - val_loss: 0.6607 - val_accuracy: 0.7798
Epoch 3/5
259/259 [==============================] - 1038s 4s/step - loss: 0.4697 - accuracy: 0.8202 - val_loss: 0.8668 - val_accuracy: 0.7323
Epoch 4/5
259/259 [==============================] - 931s 4s/step - loss: 0.4639 - accuracy: 0.8231 - val_loss: 0.6766 - val_accuracy: 0.7585
Epoch 5/5
259/259 [==============================] - 947s 4s/step - loss: 0.4271 - accuracy: 0.8391 - val_loss: 0.6361 - val_accuracy: 0.8012


In [12]:
model.save('vgg16_stewart_rotation_model.h5')

In [1]:
import os
import keras
from keras.preprocessing.image import load_img,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model,load_model

import numpy as np
import pandas as pd
from glob import glob

2022-04-01 12:39:37.800675: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-01 12:39:37.800723: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = load_model('vgg16_stewart_rotation_model.h5')

2022-04-01 12:40:19.672211: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-01 12:40:19.672248: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-01 12:40:19.672266: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2022-04-01 12:40:19.672570: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dic = {0:'0 deg',1:'180 deg',2:'270 deg',3:'90 deg',4:'unknown'}

In [11]:
path = '/home/vimalkumar/Documents/Stewart/CNN/stewart test/180 degree'

file_name = []
result = []



count = 1
for i in os.listdir(path):
    print(count ,'  ',i)
    count += 1
    img = image.load_img(os.path.join(path,i), 
                         target_size=(224,224))
    #     plt.figure(figsize=(15,20))
    #     plt.imshow(img)
    plt.show()

    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = model.predict(images)
    #print(val)

    a=np.argmax(val,axis=1)
    #print(a)
    print('Image rotated Angle : ',dic[a[0]])
    file_name.append(i)
    result.append(dic[a[0]])
    
df = pd.DataFrame({'file':file_name,'result':result})
pd.set_option('display.max_rows',1000)
display(df)

1    b14_33.png
Image rotated Angle :  0 deg
2    b14_37.png
Image rotated Angle :  0 deg
3    b8_40.png
Image rotated Angle :  180 deg
4    b8_19.png
Image rotated Angle :  0 deg
5    b8_101.png
Image rotated Angle :  180 deg
6    b14_43.png
Image rotated Angle :  unknown
7    b8_69.png
Image rotated Angle :  180 deg
8    b14_23.png
Image rotated Angle :  180 deg
9    b8_20.png
Image rotated Angle :  180 deg
10    b14_30.png
Image rotated Angle :  180 deg
11    b8_83.png
Image rotated Angle :  180 deg
12    b14_44.png
Image rotated Angle :  0 deg
13    b14_78.png
Image rotated Angle :  180 deg
14    b8_72.png
Image rotated Angle :  180 deg
15    b8_55.png
Image rotated Angle :  180 deg
16    b8_42.png
Image rotated Angle :  180 deg
17    b8_32.png
Image rotated Angle :  180 deg
18    b14_54.png
Image rotated Angle :  0 deg
19    b8_58.png
Image rotated Angle :  180 deg
20    b8_86.png
Image rotated Angle :  180 deg
21    b8_17.png
Image rotated Angle :  180 deg
22    b8_82.png
Image r

Image rotated Angle :  180 deg
175    b14_3.png
Image rotated Angle :  0 deg


,file,result
0,b14_33.png,0 deg
1,b14_37.png,0 deg
2,b8_40.png,180 deg
3,b8_19.png,0 deg
4,b8_101.png,180 deg
5,b14_43.png,unknown
6,b8_69.png,180 deg
7,b14_23.png,180 deg
8,b8_20.png,180 deg
9,b14_30.png,180 deg


In [10]:
df['result'].value_counts()

0 deg      132
180 deg     13
unknown      3
270 deg      1
Name: result, dtype: int64